## <font color='darkblue'>Preface</font>
<b><font size='3ptx'>In this tutorial, you will discover how to implement multi-head attention from scratch in TensorFlow and Keras.</font></b>

([article source](https://machinelearningmastery.com/how-to-implement-multi-head-attention-from-scratch-in-tensorflow-and-keras/)) We have already familiarised ourselves with the theory behind the [**Transformer model**](https://machinelearningmastery.com/the-transformer-model/) and its [**attention mechanism**](https://machinelearningmastery.com/the-transformer-model/), and we have already started our journey of implementing a complete model by seeing how to [implement the scaled-dot product attention](https://machinelearningmastery.com/how-to-implement-scaled-dot-product-attention-from-scratch-in-tensorflow-and-keras). **We shall now progress one step further into our journey by encapsulating the scaled-dot product attention into a multi-head attention mechanism, of which it is a core component. Our end goal remains the application of the complete model to Natural Language Processing** (NLP).

After completing this tutorial, you will know:
* The layers that form part of the multi-head attention mechanism.
* How to implement the multi-head attention mechanism from scratch.  

### <font color='darkgreen'>Tutorial Overview</font>
This tutorial is divided into three parts; they are:
1. <font size='3ptx'><b><a href='#sect1'>Recap of the Transformer Architecture</a></b></font>
    * <b><a href='#sect1_1'>The Transformer Multi-Head Attention</a></b>
2. Implementing Multi-Head Attention From Scratch
3. Testing Out the Code

### <font color='darkgreen'>Prerequisites</font>
For this tutorial, we assume that you are already familiar with:
* [The concept of attention](https://machinelearningmastery.com/what-is-attention/)
* [The Transfomer attention mechanism](https://machinelearningmastery.com/the-transformer-attention-mechanism)
* [The Transformer model](https://machinelearningmastery.com/the-transformer-model/)
* [The scaled dot-product attention](https://machinelearningmastery.com/how-to-implement-scaled-dot-product-attention-from-scratch-in-tensorflow-and-keras)

<a id='sect1'></a>
## <font color='darkblue'>Recap of the Transformer Architecture</font>
<b><font size='3ptx'>Recall having seen that the Transformer architecture follows an encoder-decoder structure</font></b>

The encoder, on the left-hand side, is tasked with mapping an input sequence to a sequence of continuous representations; the decoder, on the right-hand side, receives the output of the encoder together with the decoder output at the previous time step, to generate an output sequence ([image source: Attention is all you need](https://arxiv.org/abs/1706.03762)).

![Transformer articture](images/1.PNG)

<br/>

In generating an output sequence, the Transformer does not rely on recurrence and convolutions.

We had seen that the decoder part of the Transformer shares many similarities in its architecture with the encoder. <b>One of the core mechanisms that both the encoder and decoder share is the <font color='darkblue'>multi-head attention mechanism</font></b>. 

<a id='sect1_1'></a>
### <font color='darkgreen'>The Transformer Multi-Head Attention</font>